In [16]:
import scanpy as sc
import os
import pandas as pd

In [3]:
adata = sc.read_h5ad('../data/rna_figure_ready.h5ad')

In [8]:
adata.obs['day'].unique()

array([ 0.,  2., 12., 18., 30.])

In [9]:
raw_path = "/gpfs/gibbs/pi/krishnaswamy_smita/Chaffer_2017_mammosphere/cellranger_out/"

In [13]:
samples = [ 'HCC38_2D_culture',
 'TSA-48HR_3',
 'TSA-D12',
 'TSA-D18',
 'TSA-D30',]

In [20]:
barcodes_list = []
for sample in samples:
    barcodes_file = raw_path + sample + '/barcodes.tsv.gz'
    barcodes = pd.read_csv(barcodes_file, sep='\t', header=None, names=['barcode'])
    barcodes['sample'] = sample
    barcodes_list.append(barcodes)
barcodes_df = pd.concat(barcodes_list)

In [21]:
barcodes_df

,barcode,sample
0,AAACCTGAGAAGGTTT-1,HCC38_2D_culture
1,AAACCTGAGTTCGCGC-1,HCC38_2D_culture
2,AAACCTGCAGCTTAAC-1,HCC38_2D_culture
3,AAACCTGCAGGACCCT-1,HCC38_2D_culture
4,AAACCTGCATGTTGAC-1,HCC38_2D_culture
...,...,...
5363,TTTGTCACAGTCTTCC-1,TSA-D30
5364,TTTGTCAGTAAACACA-1,TSA-D30
5365,TTTGTCAGTACGAAAT-1,TSA-D30
5366,TTTGTCAGTACTTGAC-1,TSA-D30


In [25]:
# Create a mapping from barcode to sample
barcode_to_sample = dict(zip(barcodes_df['barcode'], barcodes_df['sample']))

# Create a new dataframe with obs_names and sample information
obs_df = pd.DataFrame(index=adata.obs_names)
obs_df['sample'] = [barcode_to_sample.get(obs_name.split('-')[0] + '-1', 'Unknown') for obs_name in adata.obs_names]
obs_df['day'] = adata.obs['day'].values

In [28]:
obs_df['sample'].unique()

array(['HCC38_2D_culture', 'TSA-D12', 'TSA-48HR_3', 'TSA-D18', 'TSA-D30'],
      dtype=object)

In [31]:
obs_df[['sample', 'day']].drop_duplicates()

,sample,day
AAACCTGAGTTCGCGC-1-4,HCC38_2D_culture,0.0
AAAGTAGAGGGTCGAT-1-4,TSA-D12,0.0
ACGATACTCAACACGT-1-4,TSA-48HR_3,0.0
AGCTCTCCATTAGCCA-1-4,TSA-D18,0.0
AGGGTGATCTGTGCAA-1-4,TSA-D30,0.0
AAACCTGGTCTAGGTT-1-5,TSA-48HR_3,2.0
AAAGATGTCACCAGGC-1-5,TSA-D12,2.0
AATCGGTAGGTTACCT-1-5,TSA-D18,2.0
ACTTTCAGTCAGAATA-1-5,TSA-D30,2.0
AAACCTGAGCAGGTCA-1-6,TSA-D12,12.0


In [38]:
obs_df[['sample', 'day']].value_counts()[obs_df[['sample', 'day']].value_counts() > 100].sort_index(level='day')

sample            day 
HCC38_2D_culture  0.0     2763
TSA-48HR_3        2.0     2769
TSA-D12           12.0    4255
TSA-D18           18.0    4012
TSA-D30           30.0    2971
dtype: int64